**Ruize Li**

Spring 2020

CS 251: Data Analysis and Visualization

Project 6: Supervised learning

In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2

## Task 3: Naive Bayes Classifier

After finishing your email preprocessing pipeline, implement the one other supervised learning algorithm we we will use to classify email, **Naive Bayes**.

### 3a) Implement Naive Bayes

In `naive_bayes.py`, implement the following methods:
- Constructor
- `train(data, y)`: Train the Naive Bayes classifier so that it records the "statistics" of the training set: class priors (i.e. how likely an email is in the training set to be spam or ham?) and the class likelihoods (the probability of a word appearing in each class — spam or ham).
- `predict(data)`: Combine the class likelihoods and priors to compute the posterior distribution. The predicted class for a test sample is the class that yields the highest posterior probability.
- `accuracy(y, y_pred)`: The usual definition :)


#### Bayes rule ingredients: Priors and likelihood (`train`)

To compute class predictions (probability that a test example belong to either spam or ham classes), we need to evaluate **Bayes Rule**. This means computing the priors and likelihoods based on the training data.

**Prior:** $$P_c = \frac{N_c}{N}$$ where $P_c$ is the prior for class $c$ (spam or ham), $N_c$ is the number of training samples that belong to class $c$ and $N$ is the total number of training samples.

**Likelihood:** $$L_{c,w} = \frac{N_{c,w} + 1}{N_{c} + M}$$ where
- $L_{c,w}$ is the likelihood that word $w$ belongs to class $c$ (*i.e. what we are solving for*)
- $N_{c,w}$ is the total count of **word $w$** in emails that are only in class $c$ (*either spam or ham*)
- $N_{c}$ is the total number of **all words** that appear in emails of the class $c$ (*total number of words in all spam emails or total number of words in all ham emails*)
- $M$ is the number of features (*number of top words*).

#### Bayes rule ingredients: Posterior (`predict`)

To make predictions, we now combine the prior and likelihood to get the posterior:

**Posterior:** $$\text{Post}_{i, c} = Log(P_c) + \sum_{j \in J_i}Log(L_{c,j})$$ where
- $\text{Post}_c$ is the posterior for class $c$ for test sample $i$(*i.e. evidence that email $i$ is spam or ham*). What we are solving for.
- $Log(P_c)$ is the logarithm of the prior for class $c$ $P_c$.
- $j \in J_i$ (under the sum) indexes the set of words in the current test sample that have nonzero counts (*i.e. which words show up in the current test set email $i$? $j$ is the index of each of these words.*)
- $\sum_{j \in J_i}Log(L_{c,j})$: we sum over the log-likelihoods ONLY PERTAINING TO CLASS $c$ at word word indices that appear in the current test email $i$ (i.e. indices at which the counts are > 0).

In [2]:
a = np.array([2,2,1,5,9, 4]).reshape((2,3))
print(np.log(a))
# print(np.where(a==0)[0])

[[0.69315 0.69315 0.     ]
 [1.60944 2.19722 1.38629]]


In [3]:
from naive_bayes_multinomial import NaiveBayes

#### Test `train`

In [4]:
num_test_classes = 4
np.random.seed(0)
data_test = np.random.random(size=(100, 6))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_test, y_test)

print(f'Your class priors are: {nbc.class_priors}\nand should be          [0.24 0.26 0.25 0.25].')
print(f'Your class likelihoods shape is {nbc.class_likelihoods.shape} and should be (4, 6).')
print(f'Your likelihoods are:\n{nbc.class_likelihoods}')


test_likelihoods = np.array([[0.15116, 0.18497, 0.17571, 0.1463 , 0.16813, 0.17374],
       [0.16695, 0.17437, 0.15742, 0.16887, 0.15677, 0.17562],
       [0.14116, 0.1562 , 0.19651, 0.17046, 0.17951, 0.15617],
       [0.18677, 0.18231, 0.15884, 0.12265, 0.16755, 0.18187]])
print(f'and should be\n{test_likelihoods}')

Your class priors are: [0.24 0.26 0.25 0.25]
and should be          [0.24 0.26 0.25 0.25].
Your class likelihoods shape is (4, 6) and should be (4, 6).
Your likelihoods are:
[[0.15116 0.18497 0.17571 0.1463  0.16813 0.17374]
 [0.16695 0.17437 0.15742 0.16887 0.15677 0.17562]
 [0.14116 0.1562  0.19651 0.17046 0.17951 0.15617]
 [0.18677 0.18231 0.15884 0.12265 0.16755 0.18187]]
and should be
[[0.15116 0.18497 0.17571 0.1463  0.16813 0.17374]
 [0.16695 0.17437 0.15742 0.16887 0.15677 0.17562]
 [0.14116 0.1562  0.19651 0.17046 0.17951 0.15617]
 [0.18677 0.18231 0.15884 0.12265 0.16755 0.18187]]


#### Test `predict`

In [5]:
num_test_classes = 4
np.random.seed(0)
data_train = np.random.random(size=(100, 10))
data_test = np.random.random(size=(4, 10))
y_test = np.random.randint(low=0, high=num_test_classes, size=(100,))

nbc = NaiveBayes(num_classes=num_test_classes)
nbc.train(data_train, y_test)
test_y_pred = nbc.predict(data_test)

print(f'Your predicted classes are {test_y_pred} and should be [2 2 2 2].')

Your predicted classes are [2 2 2 2] and should be [2 2 2 2].


### 3c) Spam filtering

Let's start classifying spam email using the Naive Bayes classifier.

- Use `np.load` to load in the train/test split that you created last week.
- Use your Naive Bayes classifier on the Enron email dataset!

**Question 9:** What accuracy do you get on the test set with Naive Bayes?

I got `89%` accuracy on the test set with Naive Bayes.

In [6]:
import email_preprocessor as epp

In [7]:
# Load your training and test data into numpy ndarrays using np.load()
# (the files you created at the end of the previous notebook)


x_train = np.load('data/email_train_x.npy')
y_train = np.load('data/email_train_y.npy')
inds_train = np.load('data/email_train_inds.npy')
x_test = np.load('data/email_test_x.npy')
y_test = np.load('data/email_test_y.npy')
inds_test = np.load('data/email_test_inds.npy')

In [8]:
# Construct your classifier
nb = NaiveBayes(2)

In [9]:
# Train and test your classifier
nb.train(x_train, y_train)
y_pred = nb.predict(x_test)
acc = nb.accuracy(y_test, y_pred)
print("accuracy is : ", acc)

accuracy is :  0.8902077151335311


### 3d) Confusion matrix

To get a better sense of the errors that the Naive Bayes classifer makes, you will create a confusion matrix. 

- Implement `confusion_matrix` in `naive_bayes.py`.
- Print out a confusion matrix of the spam classification results.

In [10]:
y_test = y_test.astype('int')
y_pred = y_pred.astype('int')
print(nb.confusion_matrix(y_test, y_pred))


[[2756.  566.]
 [ 173. 3245.]]


**Question 10:** Interpret the confusion matrix, using the convention that positive detection means spam (*e.g. a false positive means classifying a ham email as spam*). What types of errors are made more frequently by the classifier? What does this mean (*i.e. X (spam/ham) is more likely to be classified than Y (spam/ham) than the other way around*)?

**Reminder: Look back at your preprocessing code: which class indices correspond to spam/ham?**

**Answer 10:** The confusion matrix tells us that `173` spam emails are falsely classfied as `ham` and 566 `ham` emails are misclassified as `spam`. In terms of binary performance metrics, it makes `False Positive` more frequently. It means that `ham` emails are more likely to be classified as `spam`.

### 3e) Investigate the misclassification errors

Numbers are nice, but they may not the best for developing your intuition. Sometimes, you want to see what an misclassification *actually* looks like to build your understanding as you look to improve your algorithm. Here, you will take a false positive and a false negative misclassification and retrieve the actual text of the email so see which emails produced the error.

- Determine the index of the **FIRST** false positive and false negative misclassification — i.e. 2 indices in total. Remember to use your inds array to figure out the index of the emails BEFORE shuffling happened.
- **Section B:** Implement the function `retrieve_emails` in `email_preprocessor.py` to return the string of the raw email at the error indices. (**Sections A/C** have been supplied with this function on Classroom.)
- Call your function to print out the two emails that produced misclassifications.

**Question 11:** What do you think it is about each email that resulted in it being misclassified?

**Answer 11:** For the first scenario, the email has lots of numbers and words related to money, which causes the classfier to detect it as `spam`. For the second email, the message is extremely short and the classfier simply does not have enough information, and the email is easily classified incorrectly.

In [11]:
# Determine the indices of the 1st FP and FN.
# Note: spam = 0, ham = 1
inds = np.where(y_pred != y_test)[0]
# print(inds.shape)
# print(y_test[:10])

fp = inds_test[inds[np.in1d(inds, np.where(y_pred == 0)[0])][0]]
fn = inds_test[inds[np.in1d(inds, np.where(y_pred == 1)[0])][0]]
print(fp)
print(fn)

3814
28708


In [12]:
# Use retrieve_emails() to display the first FP and FN.
inds = np.array([fp, fn])
print(inds)
emails = epp.retrieve_emails(inds)

print()
print('The 1st email that is a false positive (classified as spam, but really not) is:')
print('------------------------------------------------------------------------------------------')
print(emails[0])
print('------------------------------------------------------------------------------------------')
print('The 1st email that is a false negative (classified as ham, but really spam) is:')
print('------------------------------------------------------------------------------------------')
print(emails[1])
print('------------------------------------------------------------------------------------------')

[ 3814 28708]

The 1st email that is a false positive (classified as spam, but really not) is:
------------------------------------------------------------------------------------------
Subject: hearing info

------------------------------------------------------------------------------------------
The 1st email that is a false negative (classified as ham, but really spam) is:
------------------------------------------------------------------------------------------
Subject: conoco / tw at monument - - - level " a " + / - 30 % cost estimate
design volume = 60 mmcf / d at 700 psig
the following is a summary of facility requirements and the associated cots to tie conoco to tw - 10 " suction side of the monument compressor station .
facilities a - release costs ( $ )
10 x 8 " tee & side valve 30 , 000
efm , das , rtu , chromatograh 130 , 000
and h 2 o analyzer and buildings .
8 " - ultrasonic measurements 40 , 000
flow control valve with pressure override 135 , 000
and isolation valve
ove

## Task 4) Comparison with KNN


- Run a similar analysis to what you did with Naive Bayes above. When computing accuracy on the test set, you may want to reduce the size of the test set (e.g. to the first 500 emails in the test set).
- Copy-paste your `confusion_matrix` method into `knn.py` so that you can run the same analysis on a KNN classifier.

In [13]:
from knn import KNN

In [14]:
# Construct and train your KNN classifier
knn = KNN(2)
knn.train(x_train, y_train)
print('thank you for your patience... this might take a while...')
knn_ypred = knn.predict(x_test[:500],2)

thank you for your patience... this might take a while...


In [15]:
# Evaluate the accuracy of the KNN classifier
knn_acc = knn.accuracy(y_test[:500], knn_ypred[:500])
print("knn accuracy is : ", knn_acc)

knn accuracy is :  0.912


In [16]:
num_test = 500
print(knn.confusion_matrix(y=y_test[:num_test], y_pred=y_pred))

[[208.  41.]
 [  8. 243.]]


**Question 12:** What accuracy did you get on the test set (potentially reduced in size)?

**Answer 12:** The accuracy is `91.2%`.

**Question 13:** How does the confusion matrix compare to that obtained by Naive Bayes?

**Answer 13:** The confusion matrix shows that there's fewer misclassification errors 

**Question 14:** Briefly describe at least one pro/con of KNN compared to Naive Bayes on this dataset.

**Answer 14:** KNN gives us higher accuracy but it's more computational intense and memory intense.

**Question 15:** When potentially reducing the size of the test set here, why is it important that we shuffled our train and test set?

**Answer 15:** because it we don't shuffle the data, we might end up with a substantial amount of spam data but almost zero ham data or other way around

## Extensions (Section B only)

### 1. Better text preprocessing

- If you look at the top words extracted from the email dataset, many of them are common "stop words" (e.g. a, the, to, etc.) that do not carry much meaning when it comes to differentiating between spam vs. non-spam email. Improve your preprocessing pipeline by building your top words without stop words. Analyze performance differences.

**Sources: https://www.ranks.nl/stopwords**

I created a list of common stopwords in english and implemented the improved text preprocessing by adding a conditional statement in `count_word()`. If the word exists in the stopwords list, then we ignore it.

After making the feature vector, train the classfier and predict the classes. We see that the accuracy is now `93%` which is higher than before `89%`.

So the improved method of text preprocessing indeed boosted the performance of the classifier.

In [17]:
# without stopwords
word_freq, num_emails = epp.count_words()
top_words, counts = epp.find_top_words(word_freq)

In [18]:
print(top_words[:10], counts[:10])

['the', 'to', 'and', 'of', 'a', 'in', 'for', 'you', 'is', 'this'] [290748, 213001, 157468, 148447, 118163, 106594, 84419, 82069, 71536, 63847]


In [19]:
# with stopwords
word_freq, num_emails = epp.count_words(stopwords = True)
top_words, counts = epp.find_top_words(word_freq)

In [20]:
print(top_words[:10], counts[:10])

['enron', 'subject', 'ect', 'com', 'company', 'please', 'hou', 'e', 'would', 'new'] [60909, 47811, 35346, 24185, 22959, 20350, 17264, 16018, 15529, 15272]


In [21]:
# make feature vector and divide train and test sets
# np.random.seed(0)
better_feats, better_y = epp.make_feature_vectors(top_words, num_emails)
better_x_train, better_y_train, better_train_inds, better_x_test, better_y_test, better_test_inds = epp.make_train_test_sets(better_feats, better_y)

In [22]:
nb.train(better_x_train, better_y_train)
better_ypred = nb.predict(better_x_test)
# print(y_test[:10])
# print(better_ypred[:10])
print("accuracy after text preprocessing with stopwords : ", nb.accuracy(better_y_test, better_ypred))

accuracy after text preprocessing with stopwords :  0.9267062314540059


### 2. Feature size

- Explore how the number of selected features for the email dataset influences accuracy and runtime performance.

To investigate the effect of number of features on the performance, I decided to run the Naive Bayes `find_top_words` with different number of features and see how the accuracy changes when predicting the same set of data.

This extension is very computationally intense and time-consuming. After waiting for a loooooong time, the results are presented below.

An intuitive observation is that the accuracy significantly increases as we have more features, but the downside is that the program takes longer time.

Thus it having a threshold of accuracy would give us a balance between computation time and accuracy.

In [23]:
import time
# num features selected
np.random.seed(0)
num_features = [50, 100, 200, 400, 1000]
runtime = []
accuracy = []
# run iteration
print('thank you for the patience, this might take a while...')
for num_f in num_features:
    # indicator
    print(f'running...num feature {num_f}')
    s = time.time()
    word_freq, num_emails = epp.count_words(stopwords = False)
    top_words, counts = epp.find_top_words(word_freq, num_features = num_f)
    better_feats, better_y = epp.make_feature_vectors(top_words, num_emails)
    better_x_train, better_y_train, better_train_inds, better_x_test, better_y_test, better_test_inds = epp.make_train_test_sets(better_feats, better_y)
    nb.train(better_x_train, better_y_train)
    better_ypred = nb.predict(better_x_test)
    acc = nb.accuracy(better_y_test, better_ypred)
    e = time.time()
    runtime.append(e-s)
    accuracy.append(acc)

running...num feature 50
running...num feature 100
running...num feature 200
running...num feature 400
running...num feature 1000


In [32]:
# results
print('-------------------------------')
print('num of features: ', num_features)
print('runtime(s): ', list(np.round(np.array(runtime),2)))
print('accuracy: ', list(np.round(np.array(accuracy),2)))
print('-------------------------------')

-------------------------------
num of features:  [50, 100, 200, 400, 1000]
runtime(s):  [15.48, 18.48, 24.03, 33.37, 56.82]
accuracy:  [0.77, 0.85, 0.89, 0.94, 0.96]
-------------------------------


### 3. Distance metrics
- Compare KNN performance with the $L^2$ and $L^1$ distance metrics

The results are shown below. Obviously, `l2` distance gives us higher accuracy than `l1` distance metrics. This is because we are voting based on more neighbors of the data sample. Correspondingly `l2` distance metrics will result in longer computational time.

Surprisingly, after experimenting with higher degree distance metrics, the accuracy instead goes down.

In [31]:
knn = KNN(2)
# l1 dist
knn.train(x_train, y_train)
knn_ypred = knn.predict(x_test[:200], k = 2, dist_metrics = 'l1')
print(f'l1 distance | accuracy : {knn.accuracy(y_test[:200], knn_ypred)}')
# l2 dist
knn_ypred = knn.predict(x_test[:200], k = 2, dist_metrics = 'l2')
print(f'l2 distance | accuracy : {knn.accuracy(y_test[:200], knn_ypred)}')
# l3 dist
knn_ypred = knn.predict(x_test[:200], k = 2, dist_metrics = 'l3')
print(f'l3 distance | accuracy : {knn.accuracy(y_test[:200], knn_ypred)}')

l1 distance | accuracy : 0.52
l2 distance | accuracy : 0.93
l3 distance | accuracy : 0.52


### 4. K-Fold Cross-Validation